In [ ]:
#!pip install scikit-image 

     |████████████████████████████████| 13.0 MB 3.7 MB/s            
     |████████████████████████████████| 1.9 MB 35.5 MB/s            
     |████████████████████████████████| 16.9 MB 14.6 MB/s            
     |████████████████████████████████| 33.0 MB 52.9 MB/s            
     |████████████████████████████████| 4.3 MB 50.2 MB/s            
     |████████████████████████████████| 3.0 MB 56.3 MB/s            
     |████████████████████████████████| 178 kB 61.9 MB/s            
     |████████████████████████████████| 7.3 MB 63.6 MB/s            
     |████████████████████████████████| 3.3 MB 69.4 MB/s            
     |████████████████████████████████| 880 kB 47.8 MB/s            
     |████████████████████████████████| 61 kB 52 kB/s              


In [ ]:
def make_imgcells(mask, img, background):
  imgcells = list()
  the_new_max_value =  max(mask.ravel())
  for i in range(the_new_max_value):
    print(i, end=' ')
    results = np.where(mask == the_new_max_value)
    newmask = (mask == the_new_max_value)
    newimg = np.array(img[:,:,1], copy=True)  
    newimg[np.invert(newmask)] = background
    xrange = [min(results[1]),max(results[1])]
    yrange = [min(results[0]),max(results[0])]
    if xrange[0] > 10 and xrange[1] < (mask.shape[1]-1) and yrange[0] > 10 and yrange[1] < (mask.shape[0]-1):
        imgcell = newimg[yrange[0]:yrange[1],xrange[0]:xrange[1]]
        imgcells.append(imgcell)
    mask[mask == the_new_max_value] = 0
    the_new_max_value =  max(mask.ravel())
  return imgcells

In [ ]:
def make_squarescells(imgcells, background, s=150, minsize = 60):
  #s : size of the square image:
  # minsize : for width or height - Filter to exclude small artifacts
  squarecells=list()
  i=0
  for cell in imgcells:
    #print(i,end=' ')
    i = i +1
    img = cell
    h, w = img.shape
    if h>minsize or w>minsize:
        left, right, top, down = (0,0,0,0)
        if h>s:
            top = (h-s)//2
            down = (h-s)//2
        if w>s:
            left = (w-s)//2
            right = (w-s)//2
        # creating the cropped image
        if (h-down)>s:
            down = down + 1
        if (w-right)>s:
            right = right + 1
        cimg = img[top:(h-down), left:(w-right)]
        #Creating a dark square with NUMPY  
        f = np.zeros((s,s),np.uint8)
        f.fill(background)
        #Getting the centering position
        ax,ay = (s - cimg.shape[1])//2,(s - cimg.shape[0])//2
        limx = ax+cimg.shape[1]
        limy = cimg.shape[0]+ay
        #Pasting the 'image' in a centering position
        #print(cimg.shape, ax, limx)
        f[ay:limy,ax:limx] = cimg
        squarecells.append(f)
  return squarecells

In [ ]:
from matplotlib.colors import LinearSegmentedColormap  
cmap = LinearSegmentedColormap.from_list('my_list', [(0, 0, 0), (0, 1, 0)] , N=256)

def show_images(images, columns = 8, rows = 5, titles = None, cmap = cmap):
    cmap = LinearSegmentedColormap.from_list('my_list', [(0, 0, 0), (0, 1, 0)] , N=256)
    fig=plt.figure(figsize=(columns*1.5, rows*1.5))
    for i in range(1, columns*rows +1):
        if i<=len(images):
            img = images[i-1]
            fig.add_subplot(rows, columns, i)
            plt.xticks([]), plt.yticks([]) 
            plt.imshow(img,cmap=cmap)
    plt.show()

In [ ]:
from skimage import io
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def creation_imgcells(mypat, dir_with_images, dir_with_ind_cells):
    mylist = os.listdir(dir_with_images+"/"+mypat)
    mylistpat = []
    for s in mylist:
        if '.jpg' in s: 
            mylistpat.append(s)
    
    j = 0
    for img1 in mylistpat:
        print(img1)
        img2_path = dir_with_images+"/"+mypat+ "/"+img1
        mask_path = dir_with_images+"/"+mypat+ "/Masks/MASK_" + img1[0:-4] + ".tif"

        img2 = io.imread(img2_path)
        mask = io.imread(mask_path)
        the_bkg = background = np.mean(img2[mask == 0])
        print(the_bkg)
        imgcells = make_imgcells(mask, img2, the_bkg)
        squarecells = make_squarescells(imgcells, the_bkg, s=150, minsize = 60)

        # define the name of the directory to be created
        cells_path = dir_with_ind_cells+"/"+mypat
        try:
            os.mkdir(cells_path)
        except OSError:
            print ("Creation of the directory %s failed" % cells_path)
        else:
            print ("Successfully created the directory %s " % cells_path)

        i=0
        for imagecell in squarecells:
            filename = "Cell_"+str(j)+"_"+str(i)+".png"
            i=i+1
            #print(i,end=" ")
            io.imsave(cells_path+"/"+filename, imagecell)
        j = j + 1


In [ ]:
dir_img1 = "Patterns"
dir_imgcells = "Patterns_single_cells"
import os
# mydir = os.getcwd()
mylist = os.listdir(dir_img1)
mylistpat = []
for s in mylist:
    if 'pat' in s: 
        mylistpat.append(s)
mylistpat

['pat_13',
 'pat_22',
 'pat_25',
 'pat_23',
 'pat_15',
 'pat_12',
 'pat_7',
 'pat_10',
 'pat_19',
 'pat_9',
 'pat_18',
 'pat_8',
 'pat_11',
 'pat_16']

In [ ]:
for pat in mylistpat:
  print(pat, dir_img1)
  creation_imgcells(pat, dir_img1, dir_imgcells)

pat_13 Patterns
038_0213374_1_210300885001_3_14_10-2_pat_13.jpg
5.986952409612656
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 Creation of the directory Patterns_single_cells/pat_13 failed
035_0169517_1_200102859901_3_14_07-2_pat_13.jpg
5.925639855550109
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 Creation of the directory Patterns_single_cells/pat_13 failed
052_0169486_1_200101468001_4_14_10-2_pat_13.jpg
7.02650759392486
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 Creation of the directory Patterns_single_cells/pat_13 failed
044_0213580_1_210401893101_4_14_02-4_pat_13.jpg
7.957426339904449
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32

In [ ]:
creation_imgcells('pat_13', dir_img1, dir_imgcells)

038_0213374_1_210300885001_3_14_10-2_pat_13.jpg
5.986952409612656
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 Successfully created the directory Patterns_single_cells/pat_13 
035_0169517_1_200102859901_3_14_07-2_pat_13.jpg
5.925639855550109
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 Creation of the directory Patterns_single_cells/pat_13 failed
052_0169486_1_200101468001_4_14_10-2_pat_13.jpg
7.02650759392486
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 Creation of the directory Patterns_single_cells/pat_13 failed
044_0213580_1_210401893101_4_14_02-4_pat_13.jpg
7.957426339904449
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 

In [ ]:
mylist = os.listdir(dir_img1+"/"+'pat_7')
mylistpat = []
for s in mylist:
    if '.jpg' in s: 
        mylistpat.append(s)


In [ ]:
mylistpat

['012_0169467_1_191207658201_1_14_12-3_pat_7.jpg',
 '017_0213548_1_210310652801_2_14_03-1_pat_7.jpg',
 '009_0169467_1_191207650701_1_14_09-2_pat_7.jpg',
 '004_0213525_1_210309691201_1_14_04-3_pat_7.jpg']